# 단기예보 API 구동

In [ ]:
import ssl
import pandas as pd
from datetime import datetime, timedelta
import json
import requests

#---------------------------------가장 최신 발표 시간 얻는 함수-----------------------------------------
def get_latest_forecast_time():
    # 발표 시간 목록 (24시간 형식)
    forecast_times = ["0200", "0500", "0800", "1100", "1400", "1700", "2000", "2300"]
    
    # 현재 시간
    now = datetime.now()
    
    # 현재 날짜 문자열 (YYYYMMDD)
    current_date = now.strftime("%Y%m%d")
    
    # 현재 시간 문자열 (HHMM)
    current_time = now.strftime("%H%M")
    
    # 오늘 발표된 시간 중 현재 시간보다 이전인 것 찾기
    available_times = [t for t in forecast_times if t <= current_time]
    
    # 오늘 발표된 시간 중 가장 최근 시간
    if available_times:
        latest_time = max(available_times)
        return current_date, latest_time
    else:
        # 오늘 발표된 시간이 없는 경우 (현재 시간이 0200 이전)
        # 전날 마지막 발표 시간 사용
        yesterday = now - timedelta(days=1)
        yesterday_date = yesterday.strftime("%Y%m%d")
        return yesterday_date, forecast_times[-1]  # 2300

#-----------------------------------API 호출------------------------------------------

# 날씨를 알고 싶은 시간 입력
date, time = get_latest_forecast_time()
nx = '62' # 예보 지점 x좌표
ny = '123' # 예보 지점 y좌표
numOfRows = 1000

url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0"
url2 = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"

serviceKeyE = "vf5wY%2FHyYSFgTBL63CIUXrE9QpdzM4Dklyhr%2FV3E0Zj9D7wMZSeiBfcr%2B27XEebGhg1RZAjdvDk9Rp2bF8PMwg%3D%3D"
serviceKeyD = "vf5wY/HyYSFgTBL63CIUXrE9QpdzM4Dklyhr/V3E0Zj9D7wMZSeiBfcr+27XEebGhg1RZAjdvDk9Rp2bF8PMwg=="


params = {
    'ServiceKey': serviceKeyD,
    'pageNo': '1',
    'numOfRows': numOfRows,
    'dataType': 'JSON',
    'base_date': date,
    'base_time': time,
    'nx': nx,
    'ny': ny,
}

responses = requests.get(url2, params=params)
response = json.loads(responses.text)


#-----------------------------------API 결과물들 List 로 저장------------------------------------------

section = response['response']['body']['items']['item']
dates = []
times = []
cates = []
values =[]

for i, row in enumerate(section):
    dates.append(row['fcstDate'])
    times.append(row['fcstTime'][:-2])
    cates.append(row['category'])
    values.append(row['fcstValue'])

#-----------------------------------df_api 원본 그대로 일단 표 생성------------------------------------------

df_api = pd.DataFrame({'일자':dates,
                        '시간': times,
                        '종류':cates,
                        '값':values})
df_api

#-----------------------------------Pivot_api 로 구조 정리------------------------------------------
pivot_api = df_api.pivot(index=['일자','시간'], columns='종류', values='값')

date_list = []
time_list = []
temp_list = []
rain_list = []
humid_list = []
wind_list = []
sunlight_list = []
radiation_list = []
snow_list = []

for i, row in enumerate(pivot_api.values):

    rain = pivot_api['PCP'].iloc[i]
    if rain == "강수없음":
        rain = 0
    elif rain == "1mm 미만":
        rain = 1
    elif rain == "2.0mm":
        rain = 2
    elif rain == "3.0mm":
        rain = 3

    snow = pivot_api['SNO'].iloc[i]
    if snow == "적설없음":
        snow = 0

    sky = pivot_api['SKY'].iloc[i]
    if sky == '1':  # 맑음
        sunlight = 1.0
        radiation = 3.0
    elif sky == '3':  # 구름 많음
        sunlight = 0.6
        radiation = 1.5
    elif sky == '4':  # 흐림
        sunlight = 0.2
        radiation = 0.5
    else:
        sunlight = 0.4  # 기본값
        radiation = 1.0


    date_list.append(pivot_api.index[i][0]) 
    time_list.append(pivot_api.index[i][1])
    temp_list.append(pivot_api['TMP'].iloc[i])
    rain_list.append(rain)
    humid_list.append(pivot_api['REH'].iloc[i])
    wind_list.append(pivot_api['WSD'].iloc[i])
    sunlight_list.append(sunlight)
    radiation_list.append(radiation)
    snow_list.append(snow)

#-----------------------------------df_fsct - 머신러닝에 맞는 구조로 변환------------------------------------------

df_fcst = pd.DataFrame({'일자': date_list,
                        '시간(시)': time_list,
                        '기온(℃)': temp_list, 
                        '강수량(mm)' : rain_list, 
                        '습도(%rh)' : humid_list, 
                        '풍속(m/s)' : wind_list, 
                        '일조(hr)' : sunlight_list, 
                        '일사(MJ/m2)': radiation_list,
                        '적설(cm)': snow_list})

df_fcst.head(72) # 72 시간 앞

,일자,시간(시),기온(℃),강수량(mm),습도(%rh),풍속(m/s),일조(hr),일사(MJ/m2),적설(cm)
0,20250327,12,16,0,75,2.7,0.2,0.5,0
1,20250327,13,17,0,70,3.3,0.2,0.5,0
2,20250327,14,17,0,60,3.4,0.2,0.5,0
3,20250327,15,18,0,50,3.5,0.2,0.5,0
4,20250327,16,18,0,35,4.2,0.6,1.5,0
...,...,...,...,...,...,...,...,...,...
64,20250330,12,8,0,20,2,1.0,3.0,0
65,20250330,15,8,0,30,2,1.0,3.0,0
66,20250330,18,6,0,40,2,1.0,3.0,0
67,20250330,21,3,0,50,1,1.0,3.0,0


##### API 결과물 구조
- 1시간당 12개의 아이템 존재.
- 24시간 존재

##### 시간 계산
- 하루 = 12*24 = 288 rows
- 3일 = 864 rows


- 하늘상태(SKY) 코드 : 맑음(1), 구름많음(3), 흐림(4)
- 강수형태(PTY) 코드 : (초단기) 없음(0), 비(1), 비/눈(2), 눈(3), 빗방울(5), 빗방울눈날림(6), 눈날림(7) 
                      (단기) 없음(0), 비(1), 비/눈(2), 눈(3), 소나기(4) 
- 초단기예보, 단기예보 강수량(RN1, PCP) 범주 및 표시방법(값)